processing the dataframe

In [2]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
df=pd.read_csv('Reviews.csv')
df.describe(exclude='object')

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.000000,568454.000000,568454.00000,568454.000000,5.684540e+05
mean,284227.500000,1.743817,2.22881,4.183199,1.296257e+09
std,164098.679298,7.636513,8.28974,1.310436,4.804331e+07
min,1.000000,0.000000,0.00000,1.000000,9.393408e+08
25%,142114.250000,0.000000,0.00000,4.000000,1.271290e+09
50%,284227.500000,0.000000,1.00000,5.000000,1.311120e+09
75%,426340.750000,2.000000,2.00000,5.000000,1.332720e+09
max,568454.000000,866.000000,923.00000,5.000000,1.351210e+09


In [4]:
df=df[df['Score']!=3]
def partition(x):
    if x<3:
        return 0
    return 1
df['Score']=df['Score'].map(partition)

In [5]:
df=df.sort_values(['UserId', 'Time'], ascending=True)
df.drop_duplicates(subset=['UserId','Time'], keep='first', inplace=True)
df.drop(['ProductId','UserId','ProfileName','Time','HelpfulnessNumerator','HelpfulnessDenominator'],axis=1,inplace=True)
print(df.shape)

(328732, 4)


In [6]:
df['SumTex']=df['Summary']+' . '+df['Text']
df.head()

,Id,Score,Summary,Text,SumTex
136322,136323,1,Great for the Price,I have to say I was a little apprehensive to b...,Great for the Price . I have to say I was a li...
516061,516062,1,AWESOME Coffee!!!!,Received my free K cups as a sample promotion ...,AWESOME Coffee!!!! . Received my free K cups a...
516078,516079,1,Brooklyn Bean Roastery Breakfast Blend K-Cups,Brooklyn Bean Roastery Blend K-Cups are great ...,Brooklyn Bean Roastery Breakfast Blend K-Cups ...
136544,136545,0,Less than satisfactory. I gave the Brooklyn K...,"Brooklyn ""French Roast"" K-Cup Coffee is not on...",Less than satisfactory. I gave the Brooklyn K...
83317,83318,0,"""Green"" K-cup packaging sacrifices flavor",Overall its just OK when considering the price...,"""Green"" K-cup packaging sacrifices flavor . Ov..."


Text Preprocessing remove- NaN, punctuation marks ,html tags,words with length less than 2.

In [14]:
sub_html=re.compile('<.*?>')
def clean_html(text):
    return re.sub(sub_html,' ',text)
sub_punc=re.compile('[~`@#$%^&*()_!+{}|:"<>?/.,;=\-\[\]\'\،،"\\\؟،]')
def clean_punc(text):
    return re.sub(sub_punc,'',text)

In [8]:
df[df.isnull().any(1)]

,Id,Score,Summary,Text,SumTex
33958,33959,0,NaN,I only used two maybe three tea bags and got p...,NaN


In [9]:
df.drop(index=33958,inplace =True)
df.isnull().values.any()

False

In [13]:
print(set(stopwords.words('english')))
#some important stop words that should not be excluded, mostly negative words
ss={ 'isn', 'mightn', 'no', 'needn', 'doesn', 'not', 'don', 'haven', 'ain', 'didn', 'won', 'weren'}
stop = set(stopwords.words('english')).difference(ss)

{'down', 'and', 'until', 'its', 'off', 'wouldn', 'himself', 'under', 'why', 'will', 'she', 't', 'other', 'about', 'yourselves', 'are', "mightn't", 'hers', 'over', "haven't", 'just', 'is', 'your', 'theirs', 's', "you'll", 'haven', 'where', 'between', 'the', 'them', 'as', 'who', 'after', 'both', 'should', "she's", 'can', 'does', 'this', 'ma', 'than', 'it', 'any', 'what', 'been', 'has', "shouldn't", "should've", 'into', 'itself', "won't", "weren't", 'in', "you're", 'herself', 're', 'weren', 'isn', "hadn't", 'themselves', 'while', 'there', 'again', 'same', 'which', 'when', "didn't", 'a', 'some', 'below', 'hadn', "needn't", 'won', "wouldn't", 'most', 'by', "don't", 'her', 'an', 'now', 'i', 'their', 'm', 'doing', 've', 'not', 'shan', 'was', 'him', 'up', 'so', 'having', 'you', 'd', 'further', 'yourself', 'aren', 'doesn', 'too', "shan't", 'have', "mustn't", 'above', 'ain', 'my', 'didn', 'had', 'nor', "that'll", 'for', 'to', "it's", 'they', 'those', 'were', 'each', 'don', 'during', 'ourselves',

In [15]:
sno = SnowballStemmer("english")
summary=[]
hit=miss=0
start=time.time()
for text in df['SumTex'].tolist():
#         print('TExt',type(text))
    text=text.lower()
    text=clean_html(text)
    text=word_tokenize(text)
#         print('TExt',type(text))
#         print('TExt',text)
#         text=' '
    string=[]
    for word in text:
#         print(word)
#         print('word',type(word))
        if any ([ word in stop,len(word)<3,not(word.isalpha())]):
            miss+=1
#             print(word)
            continue
        hit+=1
#         print(word)
        string.append(sno.stem(word))
#         print(string)
#         print('string',type(string))
    summary.append(string)
#     print(summary)
#     print('summary',type(summary))
print(hit,'\n',miss,'\n',(time.time()-start) )     

13374940 
 17944167 
 663.6518995761871


In [16]:
df['Cleaned_text']=summary
df.drop(['Summary','Text','Id'],axis=1,inplace=True)
df.to_csv('fine_food_data.csv', encoding='utf-8')

In [17]:
df= pd.read_csv('fine_food_data.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,Score,SumTex,Cleaned_text
0,1,Great for the Price . I have to say I was a li...,"['great', 'price', 'say', 'littl', 'apprehens'..."
1,1,AWESOME Coffee!!!! . Received my free K cups a...,"['awesom', 'coffe', 'receiv', 'free', 'cup', '..."
2,1,Brooklyn Bean Roastery Breakfast Blend K-Cups ...,"['brooklyn', 'bean', 'roasteri', 'breakfast', ..."
3,0,Less than satisfactory. I gave the Brooklyn K...,"['less', 'satisfactori', 'gave', 'brooklyn', '..."
4,0,"""Green"" K-cup packaging sacrifices flavor . Ov...","['green', 'packag', 'sacrific', 'flavor', 'ove..."


In [19]:
type(df['Cleaned_text'][0])
def tostr(x):
    return ' '.join(x)

In [20]:
count_vect = CountVectorizer(ngram_range=(1,2) ) #in scikit-learn
final_bigram_counts = count_vect.fit_transform(df['Cleaned_text'])

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
tf= vectorizer.fit_transform(df['Cleaned_text'])
tf.shape

(328731, 2824988)

BAG OF WORDS

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(final_bigram_counts,df['Score'], test_size=0.20, random_state=42)

In [23]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test, sample_weight=None)

0.9152356761525241

TF-IDF

In [24]:
X_train, X_test, y_train, y_test = train_test_split(tf,df['Score'], test_size=0.20, random_state=42)
clf = MultinomialNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test, sample_weight=None)

0.8390040610217957

In [28]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
clf = SGDClassifier( penalty='l1', loss='hinge', random_state=42)
clf.fit(X_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(X_train, y_train)
predict_y = sig_clf.predict_proba(X_test)
print( "The log loss is:",log_loss(y_test, predict_y, labels=clf.classes_, eps=1e-15))

C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\ProgramData\Anaconda3\envs\tf_gpu\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. Fr

The log loss is: 0.19449908825330123
